In [225]:
import sympy as sp
import time
import numpy as np

In [226]:
def Newton_Interger(data_x,formula,t):

    # 计算对应点的函数值
    n=len(data_x)
    y=[0]*n
    for i in range(n):
        y[i]=formula.evalf(subs={t:data_x[i]})
    # 创建差商表
    Z=[[0] *n for _ in range(n)]

    for i in range(n):
        Z[i][0]=y[i]
    # 计算差商值
    for i in range(n):
        for j in range(1,i+1):
            Z[i][j]=(Z[i][j-1]-Z[i-1][j-1])/(data_x[i]-data_x[i-j])
    # 计算插值多项式
    res=0 
    for  i in range(n):
        C=Z[i][i]
        for j in range(i):
            C*=(t-data_x[j])
        res+=C
    
    return res.expand()

In [227]:
def Hermite_Interger(data_x,formula,formula_d,t):

    start=time.time()
    # 计算对应点的函数值,并且计算对应点的导数值
    n=len(data_x)
    y=[[0]*(n) for _ in range(2)]
    for i in range(n):
            y[0][i]=formula.evalf(subs={t:data_x[i]})
            if i%2==1:
                y[1][i]=formula_d.evalf(subs={t:data_x[i]})
                
    # 创建差商表
    Z=[[0] *(n) for _ in range(n)]
    for i in range(n):
            Z[i][0]=y[0][i]
            if i%2==1:
                Z[i][1]=y[1][i]
    # 计算差商值
    for i in range(n):
        for j in range(1,i+1):
            if Z[i][j]==0:
                Z[i][j]=(Z[i][j-1]-Z[i-1][j-1])/(data_x[i]-data_x[i-j])
    
    # 计算插值多项式
    res=0
    for  i in range(n):
        C=Z[i][i]
        for j in range(i):
            C*=(t-data_x[j])
        res+=C
    end=time.time()
    print("time:",end-start)
    
    return res.expand()

In [228]:
def Romberg(formula,X,X_start,X_end,error):

    fa=0
    Fa=formula.evalf(subs={X:X_start})
    Fb=formula.evalf(subs={X:X_end})
    n=2**fa
    h=(X_end-X_start)/n
    T_single=h/2*(Fa+Fb)
    Rom=[[T_single]]

    while True:
        fa+=1
        n=2**fa
        h=(X_end-X_start)/n
        T_double=Fa+Fb
        for i in range(1,n):
            T_double+=2*formula.evalf(subs={X:X_start+i*h})
        T_double=T_double*h/2
        Rom.append([T_double])
        for i in range(1,fa+1):
            Numerator=4**(i+fa-1)
            Rom[fa].append(Numerator/(Numerator-1)*Rom[fa][i-1]-1/(Numerator-1)*Rom[fa-1][i-1])
        if abs(Rom[fa][fa]-Rom[fa][fa-1])<error:
            return Rom[fa][fa]

In [229]:
def Compute_N(a,b):

    # 构建函数
    T=sp.Symbol('t')
    formula=(a**2*sp.cos(T)**2+b**2*sp.sin(T)**2)**(1/2)
    data_t=[0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,1.1,1.2,1.3,1.4,1.5]
    formula0=Newton_Interger(formula=formula,data_x=[data_t[0],data_t[-1]],t=T)
    R0=4*Romberg(formula0,T,0,sp.pi/2,1e-6)
    print(R0.evalf())
    print(sp.latex(formula0))
    formula1=Newton_Interger(formula=formula,data_x=[data_t[0],data_t[7],data_t[-1]],t=T)
    R1=4*Romberg(formula1,T,0,sp.pi/2,1e-6)
    print(R1.evalf())
    print(sp.latex(formula1))
    formula2=Newton_Interger(formula=formula,data_x=[data_t[0],data_t[5],data_t[10],data_t[-1]],t=T)
    R2=4*Romberg(formula2,T,0,sp.pi/2,1e-6)
    print(R2.evalf())
    print(sp.latex(formula2))
    formula3=Newton_Interger(formula=formula,data_x=[data_t[0],data_t[4],data_t[8],data_t[12],data_t[-1]],t=T)
    R3=4*Romberg(formula3,T,0,sp.pi/2,1e-6)
    print(R3.evalf())
    print(sp.latex(formula3))
    formula4=Newton_Interger(formula=formula,data_x=[data_t[0],data_t[3],data_t[6],data_t[9],data_t[12],data_t[-1]],t=T)
    R4=4*Romberg(formula4,T,0,sp.pi/2,1e-6)
    print(R4.evalf())
    print(sp.latex(formula4))


In [230]:
def Compute_H(a,b):

    # 构建函数
    T=sp.Symbol('t')
    formula=(a**2*sp.cos(T)**2+b**2*sp.sin(T)**2)**(1/2)
    Diff=sp.diff(formula,T)
    data_t=[0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,1.1,1.2,1.3,1.4,1.5]

    formula0=Hermite_Interger(formula=formula,formula_d=Diff,data_x=[data_t[7],data_t[7]],t=T)
    print(sp.latex(formula0))
    R0=4*Romberg(formula0,T,0,sp.pi/2,1e-6)
    print(R0.evalf())

    formula1=Hermite_Interger(formula=formula,formula_d=Diff,data_x=[data_t[1],data_t[1],data_t[-1],data_t[-1]],t=T)
    print(sp.latex(formula1)) 
    R1=4*Romberg(formula1,T,0,sp.pi/2,1e-6)
    print(R1.evalf())
    print(sp.latex(formula1))

    formula2=Hermite_Interger(formula=formula,formula_d=Diff,data_x=[data_t[1],data_t[1],data_t[7],data_t[7],data_t[-1],data_t[-1]],t=T)
    print(sp.latex(formula2))
    R2=4*Romberg(formula2,T,0,sp.pi/2,1e-6)
    print(R2.evalf())

    formula3=Hermite_Interger(formula=formula,formula_d=Diff,data_x=[data_t[1],data_t[1],data_t[5],data_t[5],data_t[10],data_t[10],data_t[-1],data_t[-1]],t=T)
    print(sp.latex(formula3))
    R3=4*Romberg(formula3,T,0,sp.pi/2,1e-6)
    print(R3.evalf())


In [231]:
def Separte(a,b):

    T=sp.Symbol('t')
    formula=(a**2*sp.cos(T)**2+b**2*sp.sin(T)**2)**(1/2)
    data_t=[0.01*i for i in range(0,157)]
    # data_t=[0,0.3, 0.6, 0.9, 1.2 ,1.5]
    data_t.append(sp.pi/2)

    res=0
    start=time.time()
    for i in range(len(data_t)-2):
        formula=Newton_Interger(formula=formula,data_x=[data_t[i],data_t[i+1]],t=T)
        R=4*Romberg(formula,T,data_t[i],data_t[i+1],1e-6)
        res+=R.evalf()
    end=time.time()
    print('time:',end-start)
    print(res)
    res=0
    start=time.time()
    for i in range(len(data_t)-2):
        if i%2==0:
            
            formula=Newton_Interger(formula=formula,data_x=[data_t[i],data_t[i+1],data_t[i+2]],t=T)
            R=4*Romberg(formula,T,data_t[i],data_t[i+2],1e-6)
            res+=R.evalf()
    end=time.time()
    print('time:',end-start)
    print(res)
    

In [232]:
def Compute(a,b):

    print("Compute_N:")
    start=time.time()
    Compute_N(a,b)
    end=time.time()
    print("time:",end-start)
    print("Compute_H:")
    Compute_H(a,b)
    print("Separte:")
    Separte(a,b)

In [233]:
Compute(a=42.8755430086675,b=40.8633914591405)

Compute_N:
262.809203238595
42.8755430086675 - 1.33455777216032 t
263.249360179974
- 0.197863139618311 t^{2} - 1.03776306273286 t + 42.8755430086675
263.120882983195
1.17334324425772 t^{3} - 2.77315933865432 t^{2} + 0.185158936241287 t + 42.8755430086675
263.117724281299
0.0493208228814388 t^{4} + 1.00328290176454 t^{3} - 2.58833957034599 t^{2} + 0.124107277163577 t + 42.8755430086675
263.111457239217
- 0.240561236500093 t^{5} + 0.985768378629496 t^{4} - 0.280344900012167 t^{3} - 1.86692998735527 t^{2} - 0.0125137841928655 t + 42.8755430086675
time: 0.07976007461547852
Compute_H:
time: 0.00916147232055664
43.4341378770029 - 1.97425992951622 t
263.162154694173
time: 0.007005214691162109
1.09712226764628 t^{3} - 2.5974939777429 t^{2} + 0.0960341060713979 t + 42.8712289879353
263.097564206676
1.09712226764628 t^{3} - 2.5974939777429 t^{2} + 0.0960341060713979 t + 42.8712289879353
time: 0.014535665512084961
- 0.238891839094082 t^{5} + 0.975784836736168 t^{4} - 0.262367437395135 t^{3} - 1.8

In [234]:
Compute(a=41.8822717617791,b=39.0938947820857)

Compute_N:
254.028211551829
41.8822717617791 - 1.84928648448698 t
254.665572553146
- 0.286512007275404 t^{2} - 1.41951847357387 t + 41.8822717617791
254.484139231463
1.62151762674925 t^{3} - 3.84387698219295 t^{2} + 0.268114328616645 t + 41.8822717617791
254.477860006246
0.0943080162468711 t^{4} + 1.30833184980474 t^{3} - 3.51726113230916 t^{2} + 0.164568997082902 t + 41.8822717617791
254.469390213678
- 0.326833644217042 t^{5} + 1.36584271000208 t^{4} - 0.433176419991438 t^{3} - 2.53967067524118 t^{2} - 0.0202573490527148 t + 41.8822717617791
time: 0.07235074043273926
Compute_H:
time: 0.0
42.6594023854033 - 2.73026348384406 t
254.563620032887
time: 0.0
1.52097860645179 t^{3} - 3.60445937068822 t^{2} + 0.139394216250326 t + 41.8759814979469
254.434657161083
1.52097860645179 t^{3} - 3.60445937068822 t^{2} + 0.139394216250326 t + 41.8759814979469
time: 0.0166318416595459
- 0.326438390309049 t^{5} + 1.35622572487295 t^{4} - 0.409828392660869 t^{3} - 2.55652751831165 t^{2} - 0.0175296279559

In [235]:
Compute(a=40.8633914591405,b=36.5409190520717)

Compute_N:
242.607236668700
40.8633914591405 - 2.86638117883184 t
243.682081473665
- 0.483173494995709 t^{2} - 2.14162093633828 t + 40.8633914591405
243.392501871280
2.49732193093121 t^{3} - 5.95690544112286 t^{2} + 0.450002638257224 t + 40.8633914591405
243.372973859443
0.228511805288289 t^{4} + 1.76655633787284 t^{3} - 5.22823982941338 t^{2} + 0.229999462226353 t + 40.8633914591405
243.362337670081
- 0.480341507582054 t^{5} + 2.09496239149274 t^{4} - 0.785357209370995 t^{3} - 3.79924453007743 t^{2} - 0.0392298517847995 t + 40.8633914591405
time: 0.08280801773071289
Compute_H:
time: 0.015959501266479492
42.0767938078288 - 4.21327430953285 t
243.584617292394
time: 0.0
2.35924934098114 t^{3} - 5.60202808446455 t^{2} + 0.235490409983395 t + 40.8526807401515
243.259019816836
2.35924934098114 t^{3} - 5.60202808446455 t^{2} + 0.235490409983395 t + 40.8526807401515
time: 0.015643835067749023
- 0.486745227346602 t^{5} + 2.0968697609877 t^{4} - 0.758554116361567 t^{3} - 3.82603517958947 t^{2} 

In [236]:
Compute(a=40.3106783148759,b=34.9340215298776)

Compute_N:
235.686250360513
40.3106783148759 - 3.5651299960951 t
237.101355773108
- 0.636130374201576 t^{2} - 2.61093443479274 t + 40.3106783148759
236.733197455240
3.08941478015472 t^{3} - 7.40317534181156 t^{2} + 0.58844976127413 t + 40.3106783148759
236.702142598814
0.358199718275694 t^{4} + 1.96062978736462 t^{3} - 6.29827410140298 t^{2} + 0.261940085258493 t + 40.3106783148759
236.690096279236
- 0.568049732660472 t^{5} + 2.56340667812938 t^{4} - 1.05053816264383 t^{3} - 4.61522735948061 t^{2} - 0.0543238580185949 t + 40.3106783148759
time: 0.08401083946228027
Compute_H:
time: 0.0
41.8268773728709 - 5.22222435101242 t
237.035377135311
time: 0.006006002426147461
2.93542146628394 t^{3} - 6.98011415487807 t^{2} + 0.309781119148275 t + 40.2965210417782
236.518405460880
2.93542146628394 t^{3} - 6.98011415487807 t^{2} + 0.309781119148275 t + 40.2965210417782
time: 0.016523361206054688
- 0.583250598443423 t^{5} + 2.58439467305732 t^{4} - 1.03025207098703 t^{3} - 4.64669457963549 t^{2} - 0

In [237]:
Compute(a=37.2328566002455,b=30.8883836902119)

Compute_N:
213.183559858415
37.2328566002455 - 4.20632415100262 t
214.980281222950
- 0.80767766399933 t^{2} - 2.99480765500362 t + 37.2328566002455
214.533729653837
3.6142005219009 t^{3} - 8.71708414337348 t^{2} + 0.737350889780567 t + 37.2328566002455
214.486049661326
0.54223778222829 t^{4} + 1.92783175725524 t^{3} - 7.0943972447946 t^{2} + 0.267597747344512 t + 37.2328566002455
214.471643989220
- 0.613368921083349 t^{5} + 2.92012290459649 t^{4} - 1.31292246322198 t^{3} - 5.2879556105541 t^{2} - 0.0705498329507 t + 37.2328566002455
time: 0.09779858589172363
Compute_H:
time: 0.0
39.031562891089 - 6.12976243529387 t
214.993377319042
time: 0.019425392150878906
3.46419757286036 t^{3} - 8.25382712808917 t^{2} + 0.39198212282898 t + 37.214841419295
214.198052800964
3.46419757286036 t^{3} - 8.25382712808917 t^{2} + 0.39198212282898 t + 37.214841419295
time: 0.013024568557739258
- 0.644401835445139 t^{5} + 2.98042326231841 t^{4} - 1.31747132097344 t^{3} - 5.31806931483875 t^{2} - 0.0633188651